# Generate Synthetic Dataset with LLM

Reference: [Fine-Tuning Embeddings for RAG with Synthetic Data](https://medium.com/llamaindex-blog/fine-tuning-embeddings-for-rag-with-synthetic-data-e534409a3971)

Generate a synthetic dataset of (query, relevant documents) pairs from a corpus of **documents without labelers** by leveraging LLM.

## Generate Corpus

In [29]:
import json
import re
import uuid

from llama_index import SimpleDirectoryReader
from llama_index.llms import OpenAI
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import MetadataMode

# from tqdm.notebook import tqdm
from tqdm import tqdm

In [1]:
TRAIN_FILES = ['afa_docs/merkblatt-fuer-arbeitslose_ba036520.pdf']
VAL_FILES = ['afa_docs/dok_ba035980.pdf']

TRAIN_CORPUS_FPATH = 'afa_docs/train_corpus.json'
VAL_CORPUS_FPATH = 'afa_docs/val_corpus.json'

In [2]:
def load_corpus(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    if verbose:
        print(f'Loaded {len(docs)} docs')
    
    parser = SimpleNodeParser.from_defaults()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f'Parsed {len(nodes)} nodes')

    corpus = {node.node_id: node.get_content(metadata_mode=MetadataMode.NONE) for node in nodes}
    return corpus

## TODO
RANDOMIZE THE TRAIN/VAL SETS <----------------------------------------------------------

NOW IT USED A PDF FOR TRAIN AND THE OTHER FOR VALIDATION

In [9]:
train_corpus = load_corpus(TRAIN_FILES, verbose=True)
val_corpus = load_corpus(VAL_FILES, verbose=True)

Loading files ['afa_docs/merkblatt-fuer-arbeitslose_ba036520.pdf']
Loaded 103 docs


Parsing documents into nodes: 100%|██████████| 103/103 [00:00<00:00, 3039.61it/s]


Parsed 103 nodes
Loading files ['afa_docs/dok_ba035980.pdf']
Loaded 40 docs


Parsing documents into nodes: 100%|██████████| 40/40 [00:00<00:00, 3515.32it/s]

Parsed 40 nodes


In [10]:
print(f"Type: {type(train_corpus)}")
print(f"Length: {len(train_corpus)}")
for key in list(train_corpus.keys())[0:2]:
    print(train_corpus[key])
    print("-"*80)

Type: <class 'dict'>
Length: 103
49466_BA_MB_1.indd   1 10.02.2015   13:20:58Agentur für Arbeit  
Musterstadthausen  Merkblatt
1Merkblatt für
Arbeitslose 
Ihre Rechte –
Ihre Pflichten
--------------------------------------------------------------------------------
3 
Ihre Agentur für Arbeit hält eine Fülle von 
 Informationen für Sie bereit. 
Neben den Informationen in diesem Merkblatt finden 
Sie unter » www.arbeitsagentur.de  unser umfassen ­
des Online-Angebot der „eServices “ sowie ein 
 interessantes Informationsangebot aus allen Aufgaben ­
bereichen der Bundesagentur für Arbeit. Sie erhalten 
wertvolle Tipps zu den Themen Ausbil ­
dung, Berufs- und Studienwahl, Weiter ­
bildung, wichtige Informationen über 
Geldleistungen sowie ein umfangreiches 
Serviceangebot.
Über das Job- und Serviceportal  
» www.arbeitsagentur.de  können Sie beispielsweise:
•  sich arbeitsuchend und arbeitslos melden,
•  Geldleistungen, wie Arbeitslosengeld, beantragen
•  Fragen zum Arbeitslosengeld unserem

In [13]:
with open(TRAIN_CORPUS_FPATH, 'w+') as f:
    json.dump(train_corpus, f)

with open(VAL_CORPUS_FPATH, 'w+') as f:
    json.dump(val_corpus, f)

## Generate synthetic queries

Use an LLM (e.g., gpt-3.5-turbo) to generate questions using each text chunk in the corpus as context.

For both training and validation, it creates pairs (`generated question`, `text chunk as context`).These pairs are used as data points in the finetuning dataset.

In [14]:
TRAIN_QUERIES_FPATH = 'afa_docs/train_val_data/train_queries.json'
TRAIN_RELEVANT_DOCS_FPATH = 'afa_docs/train_val_data/train_relevant_docs.json'

VAL_QUERIES_FPATH = 'afa_docs/train_val_data/val_queries.json'
VAL_RELEVANT_DOCS_FPATH = 'afa_docs/train_val_data/val_relevant_docs.json'

In [15]:
with open(TRAIN_CORPUS_FPATH, 'r+') as f:
    train_corpus = json.load(f)

with open(VAL_CORPUS_FPATH, 'r+') as f:
    val_corpus = json.load(f)

In [30]:
def generate_queries(
    corpus,
    num_questions_per_chunk=2,
    prompt_template=None,
    verbose=False,
):
    """
    Automatically generate hypothetical questions that could be answered with
    doc in the corpus.
    """
    llm = OpenAI(model='gpt-3.5-turbo')

    prompt_template = prompt_template or """\
    Context information is below.
    
    ---------------------
    {context_str}
    ---------------------
    
    Given the context information and not prior knowledge.
    generate only questions based on the below query.
    
    You are a Teacher/ Professor. Your task is to setup \
    {num_questions_per_chunk} questions for an upcoming \
    quiz/examination. The questions should be diverse in nature \
    across the document. Restrict the questions to the \
    context information provided."
    """

    queries = {}
    relevant_docs = {}
    # for node_id, text in corpus.items():
    for node_id, text in tqdm(corpus.items()):
        query = prompt_template.format(context_str=text, num_questions_per_chunk=num_questions_per_chunk)
        response = llm.complete(query)
 
        result = str(response).strip().split("\n")
        questions = [
            re.sub(r"^\d+[\).\s]", "", question).strip() for question in result
        ]
        questions = [question for question in questions if len(question) > 0]
        
        for question in questions:
            question_id = str(uuid.uuid4())
            queries[question_id] = question
            relevant_docs[question_id] = [node_id]
    return queries, relevant_docs

In [43]:
train_corpus_small = dict()
i = 0
for key, value in train_corpus.items():
    train_corpus_small[key] = value
    i += 1
    if i > 10:
        break

train_corpus_small

{'38c0ae2e-65e8-4d16-8489-b894a0ae971f': '49466_BA_MB_1.indd   1 10.02.2015   13:20:58Agentur für Arbeit  \nMusterstadthausen  Merkblatt\n1Merkblatt für\nArbeitslose \nIhre Rechte –\nIhre Pflichten',
 '982d7da4-fb36-4c1b-8cea-ef09be652e06': '3 \nIhre Agentur für Arbeit hält eine Fülle von \n Informationen für Sie bereit. \nNeben den Informationen in diesem Merkblatt finden \nSie unter » www.arbeitsagentur.de  unser umfassen \xad\ndes Online-Angebot der „eServices “ sowie ein \n interessantes Informationsangebot aus allen Aufgaben \xad\nbereichen der Bundesagentur für Arbeit. Sie erhalten \nwertvolle Tipps zu den Themen Ausbil \xad\ndung, Berufs- und Studienwahl, Weiter \xad\nbildung, wichtige Informationen über \nGeldleistungen sowie ein umfangreiches \nServiceangebot.\nÜber das Job- und Serviceportal  \n» www.arbeitsagentur.de  können Sie beispielsweise:\n•  sich arbeitsuchend und arbeitslos melden,\n•  Geldleistungen, wie Arbeitslosengeld, beantragen\n•  Fragen zum Arbeitslosengeld u

In [44]:
val_corpus_small = dict()
i = 0
for key, value in val_corpus.items():
    val_corpus_small[key] = value
    i += 1
    if i > 10:
        break

val_corpus_small

{'5cd92f1c-7e1b-47c3-80ab-f70340ea1510': 'Merkblatt\nArbeitslosengeld und  \nAuslandsbeschäftigung \nDienste und Leistungen  \nder Agentur für Arbeit\nAgentur für Arbeit \nMusterstadthausen  20',
 'f25f4048-173a-4ca8-aac7-4f8dca93fccf': 'Vorwort\n2Vorwort\nDieses Merkblatt informiert Sie insbesondere über die\n•  Voraussetzungen zur Berücksichtigung von Aus \xad\nlandsbeschäftigungen und Zeiten selbstständiger \n Erwerbstätigkeit für einen Anspruch auf Arbeits\xad\nlosengeld,\n•  Voraussetzungen der Mitnahme eines Leistungsan \xad\nspruchs ins Ausland zur Arbeitsuche.\nInformationen über die übrigen allgemeinen Anspruchs \xad\nvoraussetzungen zum Bezug von Arbeitslosengeld ent \xad\nnehmen Sie bitte dem Merkblatt 1 für Arbeits  lose, das \nIhre Agentur für Arbeit für Sie bereit hält.',
 '05f83cab-b259-439c-a47b-87a13461bd51': '3Vorwort\nZum 1. Februar 2020 ist das Vereinigte Königreich \nGroßbritannien und Nordirland aus der Europäischen \nUnion ausgetreten. Das Austrittsabkommen gewäh

In [45]:
train_queries, train_relevant_docs = generate_queries(train_corpus_small)

100%|██████████| 11/11 [00:41<00:00,  3.73s/it]


In [47]:
val_queries, val_relevant_docs = generate_queries(val_corpus_small)

100%|██████████| 11/11 [00:42<00:00,  3.84s/it]


In [48]:
with open(TRAIN_QUERIES_FPATH, 'w+') as f:
    json.dump(train_queries, f)

with open(TRAIN_RELEVANT_DOCS_FPATH, 'w+') as f:
    json.dump(train_relevant_docs, f)

with open(VAL_QUERIES_FPATH, 'w+') as f:
    json.dump(val_queries, f)

with open(VAL_RELEVANT_DOCS_FPATH, 'w+') as f:
    json.dump(val_relevant_docs, f)

## Merge data

Reorganize the data for easier accessing the training and evaluation datasets

In [49]:
TRAIN_DATASET_FPATH = 'afa_docs/train_val_data/train_dataset.json'
VAL_DATASET_FPATH = 'afa_docs/train_val_data/val_dataset.json'

In [50]:
train_dataset = {
    'queries': train_queries,
    'corpus': train_corpus,
    'relevant_docs': train_relevant_docs,
}

val_dataset = {
    'queries': val_queries,
    'corpus': val_corpus,
    'relevant_docs': val_relevant_docs,
}

In [51]:
with open(TRAIN_DATASET_FPATH, 'w+') as f:
    json.dump(train_dataset, f)

with open(VAL_DATASET_FPATH, 'w+') as f:
    json.dump(val_dataset, f)